In [ ]:
# 메모리 최적화 설정 (반드시 최상단에 실행)
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True,max_split_size_mb:512'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

print("⚙️ 메모리 최적화 설정 완료")
print("PYTORCH_CUDA_ALLOC_CONF: expandable_segments:True,max_split_size_mb:512")
print("TOKENIZERS_PARALLELISM: false")
print("CUDA_LAUNCH_BLOCKING: 1")

# 🔍 한국어 텍스트 교정 모델 추론 테스트 (Google Colab)

훈련된 한국어 텍스트 교정 모델을 사용한 추론 및 성능 평가
- Google Drive에서 모델 로드
- 대화형 텍스트 교정 인터페이스
- 배치 텍스트 교정
- 성능 벤치마크

## 🚀 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi
print("\nPyTorch CUDA 지원 여부:", torch.cuda.is_available() if 'torch' in globals() else 'torch not imported yet')

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content')
print(f"현재 작업 디렉토리: {os.getcwd()}")

In [ ]:
# 필요한 패키지 설치 (양자화 지원 포함)
!pip install -q transformers datasets peft accelerate
!pip install -q gradio sentencepiece protobuf
!pip install -q evaluate rouge-score sacrebleu
!pip install -q bitsandbytes  # 양자화를 위한 패키지
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("✅ 패키지 설치 완료 (양자화 지원 포함)")

In [ ]:
# 메모리 효율성을 위한 환경 변수 설정
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# 필요한 라이브러리 임포트
import torch
import pandas as pd
import numpy as np
import time
import gc
from typing import List, Dict

from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,  # 양자화 지원
    pipeline
)
from peft import PeftModel
import gradio as gr

# GPU 메모리 정리 함수
def clear_gpu_memory():
    """GPU 메모리 정리"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

def monitor_gpu_memory():
    """GPU 메모리 사용량 모니터링"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        free = torch.cuda.mem_get_info()[0] / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        used_percent = (allocated / total) * 100
        
        print(f"🔍 GPU 메모리 상태:")
        print(f"  - 할당됨: {allocated:.2f}GB ({used_percent:.1f}%)")
        print(f"  - 예약됨: {reserved:.2f}GB")
        print(f"  - 사용가능: {free:.2f}GB")
        print(f"  - 전체: {total:.2f}GB")
        
        return {
            'allocated_gb': allocated,
            'reserved_gb': reserved,
            'free_gb': free,
            'total_gb': total,
            'used_percent': used_percent
        }
    return None

def cleanup_memory():
    """완전한 메모리 정리"""
    # 가능한 모든 모델 변수 삭제
    for var_name in ['model', 'base_model', 'tokenizer']:
        if var_name in globals():
            del globals()[var_name]
    
    clear_gpu_memory()
    print("🧹 메모리 정리 완료")
    
# 초기 메모리 정리
clear_gpu_memory()

# GPU 정보 출력
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    # 메모리 사용량 출력
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    free = torch.cuda.mem_get_info()[0] / 1024**3
    print(f"할당된 GPU 메모리: {allocated:.2f}GB")
    print(f"예약된 GPU 메모리: {reserved:.2f}GB")
    print(f"사용 가능한 GPU 메모리: {free:.2f}GB")

## 📦 모델 로드

In [ ]:
# 모델 경로 설정 (Google Drive에서)
MODEL_PATH = "/content/drive/MyDrive/korean-text-correction-model"
BASE_MODEL = "google/mt5-small"

# 모델 파일 존재 확인
print(f"📁 모델 경로: {MODEL_PATH}")
if os.path.exists(MODEL_PATH):
    print("✅ 모델 디렉토리가 존재합니다.")
    print("📋 모델 파일들:")
    !ls -la {MODEL_PATH}
else:
    print("❌ 모델 디렉토리를 찾을 수 없습니다.")
    print("먼저 colab_training.ipynb를 실행하여 모델을 훈련해주세요.")

In [ ]:
# 4-bit 양자화 설정
print("⚙️ 양자화 설정 준비...")
from transformers import BitsAndBytesConfig

# 4-bit 양자화 설정 (메모리 사용량 ~75% 감소)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

print("📊 양자화 설정:")
print(f"- 4-bit 양자화: {quantization_config.load_in_4bit}")
print(f"- 계산 타입: {quantization_config.bnb_4bit_compute_dtype}")
print(f"- 양자화 타입: {quantization_config.bnb_4bit_quant_type}")
print(f"- 이중 양자화: {quantization_config.bnb_4bit_use_double_quant}")

# 토크나이저 로드
print("\n🔤 토크나이저 로딩...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=False)

# 베이스 모델을 4-bit 양자화로 로드
print("🤖 베이스 모델 로딩 (4-bit 양자화)...")
try:
    base_model = AutoModelForSeq2SeqLM.from_pretrained(
        BASE_MODEL,
        quantization_config=quantization_config,
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )
    print("✅ 양자화된 베이스 모델 로드 성공!")
except Exception as e:
    print(f"⚠️ 양자화 로드 실패, 기본 모드로 fallback: {e}")
    base_model = AutoModelForSeq2SeqLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    )

# PEFT 모델 로드
print("🔧 PEFT 어댑터 로딩...")
model = PeftModel.from_pretrained(base_model, MODEL_PATH)

# 모델을 평가 모드로 설정
model.eval()

# 메모리 사용량 출력
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    print(f"\n📊 모델 로드 후 메모리 사용량:")
    print(f"- 할당된 GPU 메모리: {allocated:.2f}GB")
    print(f"- 예약된 GPU 메모리: {reserved:.2f}GB")

print("\n✅ 모델 로드 완료!")
print(f"모델 파라미터 수: {model.num_parameters():,}")

# 메모리 정리
clear_gpu_memory()

## 🔍 추론 클래스 정의

In [ ]:
class KoreanTextCorrector:
    """GPU 메모리 최적화된 한국어 텍스트 교정 클래스 (양자화 지원)"""
    
    def __init__(self, model, tokenizer, device, max_length=256):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.max_length = max_length
        
        # 메모리 최적화를 위한 설정
        if hasattr(self.model, 'gradient_checkpointing_enable'):
            self.model.gradient_checkpointing_enable()
        
        # 모델을 eval 모드로 설정
        self.model.eval()
        
        # 양자화 상태 확인
        is_quantized = hasattr(self.model, 'hf_quantizer') or any(
            hasattr(getattr(self.model, name), 'weight') and 
            getattr(getattr(self.model, name), 'weight').dtype == torch.uint8
            for name, _ in self.model.named_modules()
            if hasattr(getattr(self.model, name), 'weight')
        )
        
        print(f"✅ KoreanTextCorrector 초기화 완료")
        print(f"- 디바이스: {device}")
        print(f"- 최대 길이: {max_length}")
        print(f"- 양자화 상태: {'활성화' if is_quantized else '비활성화'}")
        
        # 초기 메모리 상태 출력
        monitor_gpu_memory()
    
    def correct_text(self, text: str, **generation_kwargs) -> str:
        """단일 텍스트 교정 (메모리 최적화)"""
        if not text.strip():
            return text
        
        try:
            # 메모리 정리
            clear_gpu_memory()
            
            # 입력 텍스트 전처리
            input_text = f"교정: {text.strip()}"
            
            # 토크나이제이션 (메모리 효율적)
            inputs = self.tokenizer(
                input_text,
                return_tensors="pt",
                max_length=self.max_length,
                truncation=True,
                padding=False  # 단일 텍스트이므로 padding 비활성화
            ).to(self.device)
            
            # 생성 설정 (메모리 효율적)
            default_kwargs = {
                "max_length": self.max_length,
                "num_beams": 2,  # beam 수 감소로 메모리 절약
                "temperature": 0.7,
                "do_sample": True,
                "pad_token_id": self.tokenizer.pad_token_id,
                "eos_token_id": self.tokenizer.eos_token_id,
                "no_repeat_ngram_size": 2,
                "early_stopping": True,  # 조기 종료로 메모리 절약
                "use_cache": False  # 캐시 비활성화로 메모리 절약
            }
            default_kwargs.update(generation_kwargs)
            
            # 텍스트 생성
            with torch.no_grad():
                outputs = self.model.generate(**inputs, **default_kwargs)
            
            # 결과 디코딩
            corrected = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # 메모리 정리
            del inputs, outputs
            clear_gpu_memory()
            
            return corrected.strip()
            
        except torch.cuda.OutOfMemoryError as e:
            print(f"⚠️ GPU 메모리 부족: {e}")
            clear_gpu_memory()
            # 더 작은 beam size로 재시도
            return self.correct_text_fallback(text)
        except Exception as e:
            print(f"❌ 추론 오류: {e}")
            clear_gpu_memory()
            return text
    
    def correct_text_fallback(self, text: str) -> str:
        """메모리 부족 시 폴백 방법 (양자화 환경 최적화)"""
        try:
            print("🔄 메모리 절약 모드로 재시도...")
            
            # 더 작은 최대 길이로 설정
            fallback_max_length = min(96, self.max_length // 2)
            
            input_text = f"교정: {text.strip()[:200]}"  # 입력 텍스트도 줄임
            inputs = self.tokenizer(
                input_text,
                return_tensors="pt",
                max_length=fallback_max_length,
                truncation=True,
                padding=False
            ).to(self.device)
            
            # 최소한의 설정으로 생성 (양자화 모델에 최적화)
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=fallback_max_length,
                    num_beams=1,  # greedy decoding만 사용
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    use_cache=False,
                    return_dict_in_generate=False  # 딕셔너리 반환 비활성화
                )
            
            corrected = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # 즉시 메모리 정리
            del inputs, outputs
            clear_gpu_memory()
            
            print("✅ 메모리 절약 모드 성공")
            return corrected.strip()
            
        except Exception as e:
            print(f"❌ 폴백 방법도 실패: {e}")
            print("🔄 CPU 모드로 최종 시도...")
            
            try:
                # CPU로 이동하여 최종 시도
                cpu_inputs = self.tokenizer(
                    f"교정: {text.strip()[:100]}",
                    return_tensors="pt",
                    max_length=64,
                    truncation=True,
                    padding=False
                )
                
                with torch.no_grad():
                    cpu_outputs = self.model.cpu().generate(
                        **cpu_inputs,
                        max_length=64,
                        num_beams=1,
                        do_sample=False,
                        use_cache=False
                    )
                
                result = self.tokenizer.decode(cpu_outputs[0], skip_special_tokens=True)
                
                # 모델을 다시 GPU로 이동
                self.model.to(self.device)
                clear_gpu_memory()
                
                return result.strip()
                
            except Exception as cpu_e:
                print(f"❌ CPU 모드도 실패: {cpu_e}")
                clear_gpu_memory()
                return text
    
    def correct_batch(self, texts: List[str], batch_size: int = 1, **generation_kwargs) -> List[str]:
        """배치 텍스트 교정 (양자화 환경 최적화)"""
        results = []
        total_batches = (len(texts) + batch_size - 1) // batch_size
        
        print(f"📝 배치 처리 시작: {len(texts)}개 텍스트, 배치 크기: {batch_size}")
        
        # 양자화 모델에서는 배치 크기를 더 작게 설정
        safe_batch_size = min(batch_size, 2)  # 양자화 환경에서는 최대 2개
        
        for i in range(0, len(texts), safe_batch_size):
            batch_texts = texts[i:i + safe_batch_size]
            batch_num = i // safe_batch_size + 1
            
            print(f"🔄 배치 {batch_num}/{total_batches} 처리 중...")
            
            # 배치 시작 전 메모리 상태 확인
            if torch.cuda.is_available():
                memory_info = monitor_gpu_memory()
                if memory_info and memory_info['used_percent'] > 85:
                    print("⚠️ 메모리 사용량이 높습니다. 정리 중...")
                    clear_gpu_memory()
            
            # 각 텍스트를 개별적으로 처리 (메모리 안전)
            batch_results = []
            for j, text in enumerate(batch_texts):
                try:
                    print(f"  - 텍스트 {j+1}/{len(batch_texts)} 처리 중...")
                    corrected = self.correct_text(text, **generation_kwargs)
                    batch_results.append(corrected)
                    
                    # 각 텍스트 처리 후 즉시 메모리 정리
                    clear_gpu_memory()
                    
                except Exception as e:
                    print(f"  ❌ 텍스트 처리 실패: {e}")
                    batch_results.append(text)  # 원본 텍스트 반환
                    clear_gpu_memory()
            
            results.extend(batch_results)
            
            # 배치 완료 후 메모리 정리
            clear_gpu_memory()
            
            print(f"✅ 배치 {batch_num} 완료 ({len(batch_results)}개 처리)")
        
        print(f"🎉 전체 배치 처리 완료: {len(results)}개 결과")
        return results
    
    def benchmark_correction(self, text: str, num_runs: int = 3) -> Dict[str, Any]:
        """양자화 모델의 메모리 효율적인 성능 벤치마크"""
        print(f"🔬 벤치마크 시작: {num_runs}회 실행")
        
        times = []
        memory_usage = []
        corrected_text = ""
        
        # 초기 메모리 상태
        initial_memory = monitor_gpu_memory()
        
        for i in range(num_runs):
            print(f"  실행 {i+1}/{num_runs}...")
            
            # 실행 전 메모리 정리
            clear_gpu_memory()
            
            # 메모리 측정 시작
            start_memory = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0
            
            # 시간 측정
            start_time = time.time()
            corrected_text = self.correct_text(text)
            end_time = time.time()
            
            # 메모리 측정 종료
            end_memory = torch.cuda.memory_allocated(0) / 1024**3 if torch.cuda.is_available() else 0
            memory_used = end_memory - start_memory
            
            times.append(end_time - start_time)
            memory_usage.append(memory_used)
            
            # 각 실행 후 메모리 정리
            clear_gpu_memory()
        
        # 최종 메모리 상태
        final_memory = monitor_gpu_memory()
        
        result = {
            "original": text,
            "corrected": corrected_text,
            "performance": {
                "avg_time": np.mean(times),
                "min_time": np.min(times),
                "max_time": np.max(times),
                "std_time": np.std(times),
                "avg_memory_mb": np.mean(memory_usage) * 1024,
                "max_memory_mb": np.max(memory_usage) * 1024
            },
            "memory_info": {
                "initial_allocated_gb": initial_memory['allocated_gb'] if initial_memory else 0,
                "final_allocated_gb": final_memory['allocated_gb'] if final_memory else 0,
                "memory_efficiency": "양자화된 모델 사용" if hasattr(self.model, 'hf_quantizer') else "일반 모델"
            }
        }
        
        print(f"✅ 벤치마크 완료:")
        print(f"  - 평균 시간: {result['performance']['avg_time']:.3f}초")
        print(f"  - 평균 메모리 사용: {result['performance']['avg_memory_mb']:.1f}MB")
        
        return result

print("✅ 메모리 최적화된 KoreanTextCorrector 클래스 정의 완료")

## 🚀 양자화 모델 추론기 생성 및 테스트

In [ ]:
# 양자화된 모델로 추론기 생성
print("🤖 양자화 텍스트 교정기 생성 중...")

# 모델과 토크나이저가 로드되었는지 확인
if 'model' not in globals() or 'tokenizer' not in globals():
    print("❌ 모델이 로드되지 않았습니다. 먼저 위의 모델 로드 셀을 실행해주세요.")
else:
    # 추론기 생성 전 메모리 상태 확인
    print("\n📊 추론기 생성 전 메모리 상태:")
    pre_corrector_memory = monitor_gpu_memory()
    
    # 텍스트 교정기 생성
    corrector = KoreanTextCorrector(
        model=model,
        tokenizer=tokenizer,
        device=device,
        max_length=256  # 양자화 환경에서 적절한 길이
    )
    
    print("\n📊 추론기 생성 후 메모리 상태:")
    post_corrector_memory = monitor_gpu_memory()
    
    # 메모리 절약 효과 계산
    if pre_corrector_memory and post_corrector_memory:
        memory_increase = post_corrector_memory['allocated_gb'] - pre_corrector_memory['allocated_gb']
        print(f"\n💾 추론기 생성으로 인한 메모리 증가: {memory_increase:.3f}GB")
    
    print("\n🎉 양자화 텍스트 교정기 생성 완료!")

In [ ]:
# 양자화 상태 검증
print("🔍 양자화 상태 검증")
print("=" * 50)

# 모델의 양자화 상태 확인
try:
    # BitsAndBytesConfig 사용 여부 확인
    is_quantized = False
    quantization_info = {}
    
    if hasattr(model, 'hf_quantizer'):
        is_quantized = True
        quantization_info['quantizer_type'] = type(model.hf_quantizer).__name__
        if hasattr(model.hf_quantizer, 'quantization_config'):
            config = model.hf_quantizer.quantization_config
            quantization_info['load_in_4bit'] = getattr(config, 'load_in_4bit', False)
            quantization_info['compute_dtype'] = getattr(config, 'bnb_4bit_compute_dtype', None)
            quantization_info['quant_type'] = getattr(config, 'bnb_4bit_quant_type', None)
    
    # 레이어별 양자화 상태 확인
    quantized_layers = 0
    total_layers = 0
    
    for name, module in model.named_modules():
        if hasattr(module, 'weight'):
            total_layers += 1
            if hasattr(module.weight, 'dtype'):
                if module.weight.dtype in [torch.uint8, torch.int8]:
                    quantized_layers += 1
    
    print(f"✅ 양자화 상태: {'활성화' if is_quantized else '비활성화'}")
    if quantization_info:
        print(f"📊 양자화 정보:")
        for key, value in quantization_info.items():
            print(f"  - {key}: {value}")
    
    print(f"🔢 양자화된 레이어: {quantized_layers}/{total_layers}")
    
    if quantized_layers > 0:
        quantization_ratio = (quantized_layers / total_layers) * 100
        print(f"📈 양자화 비율: {quantization_ratio:.1f}%")
        
        # 예상 메모리 절약량 계산
        memory_saving = quantization_ratio * 0.75  # 대략적인 절약량
        print(f"💾 예상 메모리 절약: ~{memory_saving:.1f}%")
    
except Exception as e:
    print(f"❌ 양자화 상태 확인 중 오류: {e}")

print("\n" + "=" * 50)

## 🧪 기본 테스트

In [ ]:
# 간단한 테스트 케이스
test_cases = [
    "안녕하셰요",              # 안녕하세요
    "감사햐니다",              # 감사합니다
    "잘 뫃겠습니다",           # 잘 부탁드립니다
    "괜챠습니까",              # 괜찮습니까
    "이거 어떻게 생간하세요",   # 이거 어떻게 생각하세요
    "오늘 날시가 좋네요",       # 오늘 날씨가 좋네요
    "회의는 몇시에 시작해나요",  # 회의는 몇시에 시작합니까
    "점심 메뉴는 뭐가 조을까요" # 점심 메뉴는 뭐가 좋을까요
]

print("🧪 기본 텍스트 교정 테스트")
print("=" * 60)

for i, test_text in enumerate(test_cases, 1):
    start_time = time.time()
    corrected = corrector.correct_text(test_text)
    end_time = time.time()
    
    print(f"{i:2d}. 원본: {test_text}")
    print(f"    교정: {corrected}")
    print(f"    시간: {(end_time - start_time)*1000:.1f}ms")
    print("-" * 50)

## ⚡ 성능 벤치마크

In [ ]:
# 성능 벤치마크
benchmark_texts = [
    "안녕하셰요",
    "이거 정말 맛있게 생겼네요. 어디서 살 수 있나요?",
    "회의는 내일 오전 10시에 시작할 예정입니다. 참석 가능하신지 확인 부탁드려요."
]

print("⚡ 성능 벤치마크 테스트")
print("=" * 60)

for i, text in enumerate(benchmark_texts, 1):
    print(f"\n{i}. 벤치마크 텍스트: {text[:30]}...")
    
    benchmark_result = corrector.benchmark_correction(text, num_runs=5)
    
    print(f"   원본: {benchmark_result['original']}")
    print(f"   교정: {benchmark_result['corrected']}")
    print(f"   평균 시간: {benchmark_result['performance']['avg_time']*1000:.1f}ms")
    print(f"   최소 시간: {benchmark_result['performance']['min_time']*1000:.1f}ms")
    print(f"   최대 시간: {benchmark_result['performance']['max_time']*1000:.1f}ms")
    print(f"   표준편차: {benchmark_result['performance']['std_time']*1000:.1f}ms")
    print(f"   평균 메모리: {benchmark_result['performance']['avg_memory_mb']:.1f}MB")
    print(f"   최대 메모리: {benchmark_result['performance']['max_memory_mb']:.1f}MB")
    print(f"   효율성: {benchmark_result['memory_info']['memory_efficiency']}")

## 📊 배치 처리 테스트

In [ ]:
# 배치 처리 테스트
batch_texts = [
    "안녕하셰요",
    "감사햐니다", 
    "잘 뫃겠습니다",
    "괜챠습니까",
    "이거 어떻게 생간하세요",
    "오늘 날시가 좋네요",
    "회의는 몇시에 시작해나요",
    "점심 메뉴는 뭐가 조을까요",
    "이 문서를 검토해 주세요",
    "프로젝트 진행 상황은 어떤가요"
]

print("📊 배치 처리 테스트 (양자화 모델)")
print(f"총 {len(batch_texts)}개 텍스트 처리")
print("=" * 60)

# 배치 처리 전 메모리 상태
print("\n🔍 배치 처리 전 메모리 상태:")
pre_batch_memory = monitor_gpu_memory()

start_time = time.time()
batch_results = corrector.correct_batch(batch_texts, batch_size=1)  # 양자화 환경에서 안전한 배치 크기
end_time = time.time()

# 배치 처리 후 메모리 상태
print("\n🔍 배치 처리 후 메모리 상태:")
post_batch_memory = monitor_gpu_memory()

total_time = end_time - start_time
avg_time_per_text = total_time / len(batch_texts)

print(f"\n📈 배치 처리 결과:")
for i, (original, corrected) in enumerate(zip(batch_texts, batch_results), 1):
    print(f"{i:2d}. {original} → {corrected}")

print(f"\n⏱️ 처리 시간:")
print(f"전체 시간: {total_time:.2f}초")
print(f"평균 시간: {avg_time_per_text*1000:.1f}ms/텍스트")
print(f"처리 속도: {len(batch_texts)/total_time:.1f}텍스트/초")

# 메모리 효율성 분석
if pre_batch_memory and post_batch_memory:
    memory_change = post_batch_memory['allocated_gb'] - pre_batch_memory['allocated_gb']
    print(f"\n💾 메모리 효율성:")
    print(f"메모리 변화: {memory_change:+.3f}GB")
    print(f"최대 메모리 사용률: {max(pre_batch_memory['used_percent'], post_batch_memory['used_percent']):.1f}%")
    
    if memory_change < 0.1:  # 100MB 미만 증가
        print("✅ 메모리 효율적 처리 성공!")
    else:
        print("⚠️ 메모리 사용량 증가 감지")

## 🖥️ 대화형 인터페이스 (Gradio)

In [ ]:
def gradio_correct_text(text, num_beams, temperature, do_sample):
    """Gradio용 텍스트 교정 함수"""
    if not text.strip():
        return "텍스트를 입력해주세요.", ""
    
    try:
        start_time = time.time()
        
        corrected = corrector.correct_text(
            text,
            num_beams=int(num_beams),
            temperature=float(temperature),
            do_sample=bool(do_sample)
        )
        
        end_time = time.time()
        processing_time = f"처리 시간: {(end_time - start_time)*1000:.1f}ms"
        
        return corrected, processing_time
        
    except Exception as e:
        return f"오류 발생: {str(e)}", ""

# Gradio 인터페이스 생성
with gr.Blocks(title="한국어 텍스트 교정기") as demo:
    gr.Markdown("# 🇰🇷 한국어 텍스트 교정기")
    gr.Markdown("mT5 기반 한국어 텍스트 교정 모델로 맞춤법과 문법을 교정합니다.")
    
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="교정할 텍스트",
                placeholder="교정하고 싶은 한국어 텍스트를 입력하세요...",
                lines=5
            )
            
            with gr.Row():
                num_beams = gr.Slider(
                    minimum=1, maximum=8, value=4, step=1,
                    label="Beam Search 크기"
                )
                temperature = gr.Slider(
                    minimum=0.1, maximum=2.0, value=0.7, step=0.1,
                    label="Temperature"
                )
                do_sample = gr.Checkbox(value=True, label="Sampling 사용")
            
            correct_btn = gr.Button("교정하기", variant="primary")
            
        with gr.Column():
            output_text = gr.Textbox(
                label="교정된 텍스트",
                lines=5
            )
            processing_info = gr.Textbox(
                label="처리 정보",
                lines=1
            )
    
    # 예시 텍스트
    gr.Markdown("### 📝 예시 텍스트")
    examples = gr.Examples(
        examples=[
            ["안녕하셰요"],
            ["감사햐니다"],
            ["잘 뫃겠습니다"],
            ["이거 어떻게 생간하세요"],
            ["오늘 날시가 정말 좋네요"]
        ],
        inputs=[input_text]
    )
    
    # 이벤트 연결
    correct_btn.click(
        fn=gradio_correct_text,
        inputs=[input_text, num_beams, temperature, do_sample],
        outputs=[output_text, processing_info]
    )

# 인터페이스 실행
print("🖥️ Gradio 인터페이스를 시작합니다...")
demo.launch(share=True, debug=True)

## 📈 모델 평가 (선택사항)

In [ ]:
# 평가용 데이터 로드 (Google Drive에서)
def evaluate_on_test_data():
    """테스트 데이터로 모델 평가"""
    
    # 테스트 데이터 파일 경로 (예시)
    test_file_path = "/content/drive/MyDrive/구어체_대화체_16878_sample_난독화결과.csv"
    
    if not os.path.exists(test_file_path):
        print("❌ 테스트 데이터 파일을 찾을 수 없습니다.")
        return
    
    print("📊 테스트 데이터로 모델 평가 중...")
    
    # 데이터 로드
    df = pd.read_csv(test_file_path, encoding='utf-8')
    
    # 작은 샘플로 테스트
    test_df = df.sample(n=50, random_state=42)
    
    predictions = []
    references = []
    
    print(f"총 {len(test_df)}개 샘플 평가 중...")
    
    for idx, row in test_df.iterrows():
        original = row['original']
        error_text = row['obfuscated']
        
        # 모델 예측
        predicted = corrector.correct_text(error_text)
        
        predictions.append(predicted)
        references.append(original)
        
        if len(predictions) % 10 == 0:
            print(f"진행률: {len(predictions)}/{len(test_df)}")
    
    # ROUGE 점수 계산
    rouge = evaluate.load("rouge")
    rouge_scores = rouge.compute(
        predictions=predictions,
        references=references
    )
    
    print("\n📈 평가 결과:")
    print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
    print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
    print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")
    
    # 몇 가지 예시 출력
    print("\n📝 예시 결과:")
    for i in range(min(5, len(predictions))):
        print(f"\n{i+1}.")
        print(f"오류문: {test_df.iloc[i]['obfuscated']}")
        print(f"정답: {references[i]}")
        print(f"예측: {predictions[i]}")
        print("-" * 40)

# 평가 실행 (선택사항)
# evaluate_on_test_data()

## 🧹 정리
```

In [ ]:
# 🔬 양자화 모델 최종 테스트

print("🔬 양자화 모델 최종 성능 테스트")
print("=" * 60)

# 1. 단일 텍스트 교정 테스트
print("\n1️⃣ 단일 텍스트 교정 테스트:")
test_text = "안녕하셰요! 오늘 날시가 정말 좋네요."
result = corrector.correct_text(test_text)
print(f"원본: {test_text}")
print(f"교정: {result}")

# 2. 메모리 효율성 테스트
print("\n2️⃣ 메모리 효율성 테스트:")
pre_memory = monitor_gpu_memory()

# 여러 번의 추론으로 메모리 누수 확인
for i in range(5):
    _ = corrector.correct_text(f"테스트 {i+1}: 안녕하셰요")
    
print("\n5회 추론 후 메모리 상태:")
post_memory = monitor_gpu_memory()

if pre_memory and post_memory:
    memory_change = post_memory['allocated_gb'] - pre_memory['allocated_gb']
    print(f"\n메모리 변화: {memory_change:+.3f}GB")
    if abs(memory_change) < 0.1:
        print("✅ 메모리 누수 없음 - 안정적!")
    else:
        print("⚠️ 메모리 사용량 변화 감지")

# 3. 양자화 상태 최종 확인
print("\n3️⃣ 양자화 상태 최종 확인:")
is_quantized = hasattr(model, 'hf_quantizer')
print(f"양자화 활성화: {'✅ 예' if is_quantized else '❌ 아니오'}")

if is_quantized:
    print("🎉 4-bit 양자화가 성공적으로 적용되었습니다!")
    print("💾 예상 메모리 절약: ~75%")
    print("⚡ GPU 메모리 부족 문제가 해결되었습니다!")
else:
    print("⚠️ 양자화가 적용되지 않았습니다.")
    print("🔧 Fallback 모드에서 실행 중입니다.")

print("\n" + "=" * 60)
print("🎊 양자화 기반 한국어 텍스트 교정 시스템 준비 완료!")
print("이제 메모리 부족 없이 안전하게 사용할 수 있습니다.")

## 📖 사용 가이드

### 🚀 빠른 시작:

**1. 단일 텍스트 교정:**
```python
result = corrector.correct_text("안녕하셰요")
print(result)
```

**2. 배치 텍스트 교정:**
```python
texts = ["안녕하셰요", "감사햐니다", "잘 뫃겠습니다"]
results = corrector.correct_batch(texts, batch_size=1)
for original, corrected in zip(texts, results):
    print(f"{original} → {corrected}")
```

**3. 성능 벤치마크:**
```python
benchmark = corrector.benchmark_correction("테스트 텍스트", num_runs=3)
print(f"평균 시간: {benchmark['performance']['avg_time']*1000:.1f}ms")
```

**4. 메모리 모니터링:**
```python
monitor_gpu_memory()  # 현재 메모리 상태 확인
clear_gpu_memory()    # 메모리 정리
```

### 🔧 문제 해결:

**메모리 부족 시:**
1. `clear_gpu_memory()` 실행
2. 배치 크기를 1로 감소
3. 텍스트 길이 축소 (256자 이하 권장)

**성능 향상 팁:**
- Beam search 크기: 1-2 (기본값: 2)
- Temperature: 0.7 (기본값)
- 배치 크기: 1 (양자화 환경 권장)

### 🎯 양자화 효과:
- ✅ GPU 메모리 사용량 ~75% 감소
- ✅ OutOfMemoryError 해결
- ✅ 안정적인 장시간 사용 가능
- ⚠️ 약간의 품질 저하 가능 (일반적으로 미미함)

이제 **메모리 효율적인 한국어 텍스트 교정**을 즐기세요! 🎉
```